In [2]:
import pandas as pd
import os
import json
import pymysql
from pyecharts.charts import Graph
from pyecharts import options as opts
from pyecharts.globals import ThemeType
import re
from dominate.tags import *
import dominate
import difflib
from pyecharts.commons.utils import JsCode

In [3]:
db = pymysql.connect(host='localhost',
                     user='',
                     password='',
                     database='public')
cursor = db.cursor()

In [4]:
lang_map = {}
line_map = {}
with open("zh_CN.lang", "r", encoding="utf-8") as lang:
    for i in lang:
        i = re.sub("§.", "", i)
        i = i.replace("%n", "")
        if "gtnh.line" in i:
            temp = i.split("=", maxsplit=1)
            line_map[temp[0].replace("gtnh.line", "")] = temp[1].strip()
        elif "name" in i or "desc" in i:
            temp = i.split("=", maxsplit=1)
            lang_map[temp[0].replace("gtnh.quest", "")] = temp[1].strip()

In [5]:
df = pd.DataFrame(columns=["quest_file_name", "quest_id",
                  "quest_name", "quest_desc", "quest_icon", "pre_quests"])
for quest_line in os.listdir("DefaultQuests/Quests/"):
    for quest in os.listdir(f"DefaultQuests/Quests/{quest_line}/"):
        que = json.load(
            open(f"DefaultQuests/Quests/{quest_line}/{quest}", "r", encoding="utf-8"))
        
        pre_quests = []
        for i in que["preRequisites:9"]:
            pre_quests.append(que["preRequisites:9"][i]["questIDLow:4"])
        
        item_id = "i~" + que["properties:10"]['betterquesting:10']["icon:10"]["id:8"].replace(":", "~") + "~" + str(que["properties:10"]['betterquesting:10']["icon:10"]["Damage:2"])
        cursor.execute(f"select ID, INTERNAL_NAME, LOCALIZED_NAME from item where ID like '%{item_id}%'")
        icon_name = ""
        try:
            icon_name = cursor.fetchall()[0][2].encode(
                "utf-8").decode("unicode_escape")
        except:
            ...
        df.loc[len(df)] = [quest, que["questIDLow:4"], que["properties:10"]
                           ["betterquesting:10"]["name:8"], que["properties:10"]["betterquesting:10"]["desc:8"], icon_name, pre_quests]

counter = 1
for i in df[df["quest_name"] == "AE Spatial Storage"].sort_values("quest_id").index:
    df.loc[i, "quest_name"] = df.loc[i, "quest_name"] + str(counter)
    counter += 1

df["quest_name_zh"] = df.apply(lambda x: lang_map[str(
    x["quest_id"])+".name"] if str(x["quest_id"])+".name" in lang_map.keys() else "", axis=1)
df["quest_desc_zh"] = df.apply(lambda x: lang_map[str(
    x["quest_id"])+".desc"] if str(x["quest_id"])+".desc" in lang_map.keys() else "", axis=1)

counter = 1
for i in df[df["quest_name_zh"] == "AE空间存储"].sort_values("quest_id").index:
    df.loc[i, "quest_name_zh"] = df.loc[i, "quest_name_zh"] + str(counter)
    counter += 1

counter = 1
for i in df[df["quest_name_zh"] == "自动合成"].sort_values("quest_id").index:
    df.loc[i, "quest_name_zh"] = df.loc[i, "quest_name_zh"] + str(counter)
    counter += 1

In [23]:
for questline in os.listdir("DefaultQuests/QuestLines"):
    nodes = []
    links = []
    for quest in os.listdir(f"DefaultQuests/QuestLines/{questline}"):
        if quest == "QuestLine.json":
            continue
        que = json.load(
            open(f"DefaultQuests/QuestLines/{questline}/{quest}", "r", encoding="utf-8"))
        line = df[df["quest_id"] == que["questIDLow:4"]]

        desc = line["quest_desc"].values[0]
        if len(line["quest_desc_zh"].values[0]) != 0:
            desc = line["quest_desc_zh"].values[0]

        temp_list = []
        for i in range(0, len(desc), 50):
            temp_list.append(desc[i : i + 50])

        if os.path.exists(f"html/2.7.0全物品图标/2.7.0/{line["quest_icon"].values[0]}.png"):
            path = f"image://2.7.0全物品图标/2.7.0/{line["quest_icon"].values[0]}.png"
        else:
            path = ""
        
        name = line["quest_name"].values[0]
        if len(line["quest_name_zh"].values[0]) != 0:
            name = line["quest_name_zh"].values[0]

        new_node = {"name": name,
                        "symbolSize": int(que["sizeX:3"]) * 1.3,
                        "symbol": path,
                        "x": (que["x:3"] + int(que["sizeX:3"]) / 2),
                        "y": (que["y:3"] + int(que["sizeX:3"]) / 2),
                        "tooltip": "<br/>".join(temp_list)
                        }
        nodes.append(new_node)
        pre_quests = line["pre_quests"].values[0]
        for quest_id in pre_quests:
            pre_que = df[df["quest_id"] == quest_id]

            pre_name = pre_que["quest_name"].values[0]
            if len(pre_que["quest_name_zh"].values[0]) != 0:
                pre_name = pre_que["quest_name_zh"].values[0]

            links.append({"source": pre_name,
                          "target": name})

    g = (
        Graph(init_opts=opts.InitOpts(width="100%",
              height="900%", bg_color="#f5f0d3", chart_id="this_chart"))
        .add("", nodes, links, is_draggable=False, edge_symbol=['circle', 'arrow'], repulsion=0, label_opts=opts.LabelOpts(position="up", is_show=False), gravity=0, layout="none", edge_symbol_size=10)
        .add_js_funcs(
        """
            chart_this_chart.on('click', function (params) {
                    if (params.dataType === 'node') {
                        navigator.clipboard.writeText(params.data.tooltip.replaceAll("<br/>", ""));
                    }
            });
        """
        )
        .render(f"html/{questline.split("-")[0]}.html")
    )

In [16]:
temp = []
for i in nodes:
    if i["name"] == "自动合成":
        print(i)

{'name': '自动合成', 'symbolSize': 31.200000000000003, 'symbol': 'image://2.7.0全物品图标/2.7.0/合成单元.png', 'x': 396.0, 'y': 192.0, 'tooltip': '这是合成处理单元的组成部分之一,这个特殊的方块并没有提供合成处理单元的功能,但是可以作为一个"填充"<br/>块存在. 并且也是制造合成监视器、并行处理器和不同规格的合成存储器的基础组件.'}
{'name': '自动合成', 'symbolSize': 31.200000000000003, 'symbol': 'image://2.7.0全物品图标/2.7.0/1k合成存储器.png', 'x': 396.0, 'y': 228.0, 'tooltip': '这是AE的核心功能,让我们开始吧. 装好合成存储器和分子装配室以及样板终端,然后在分子装配室贴上ME<br/>接口(如果接口是面板形态,请确保分子装配室另一侧有供能,当然绝大多数情况下我们在这里都会使用方块形态<br/>的接口). 如果不配合ME接口使用,分子装配室内部只有一个存储样板的空间. 将空白样板放入样板终端,<br/>然后在合成栏编写样板,最后点击箭头存储. 将编写好的样板放入ME接口后,你就可以在终端直接下单合成了<br/>.至于加工样板(机器或是其他不同于传统3x3合成),需要点击右侧的按钮切换. 且需要同时标记原料与产<br/>物. 比如,可以将ME接口贴到熔炉上,结合ME输入总线用于将产物送回ME网络. 如果是能够自动输出的<br/>机器(比如多方块机器),只需要在输出口贴上ME接口即可(这样更快且还没有什么延迟).'}


In [61]:
quest_line_order = []
for line in open("DefaultQuests/QuestLinesOrder.txt").readlines():
    quest_line_order.append(line.split(": ")[1].strip())

In [74]:
len(quest_line_order)

47

In [75]:
a = ul()
    # if "html" in i:
    #     a += li(button(i.split(".html")[0], id=i))
for que_line in quest_line_order:
    q = difflib.get_close_matches(que_line, os.listdir("html"))[0]
    a += li(button(q.split(".html")[0], id=q))
print(a)

<ul>
  <li>
    <button id="AndSoItBegins.html">AndSoItBegins</button>
  </li>
  <li>
    <button id="TipsandTricks.html">TipsandTricks</button>
  </li>
  <li>
    <button id="Tier0StoneAge.html">Tier0StoneAge</button>
  </li>
  <li>
    <button id="Tier05Steam.html">Tier05Steam</button>
  </li>
  <li>
    <button id="Tier1LV.html">Tier1LV</button>
  </li>
  <li>
    <button id="Tier2MV.html">Tier2MV</button>
  </li>
  <li>
    <button id="Tier3HV.html">Tier3HV</button>
  </li>
  <li>
    <button id="Tier4EV.html">Tier4EV</button>
  </li>
  <li>
    <button id="Tier5IV.html">Tier5IV</button>
  </li>
  <li>
    <button id="Tier6LuV.html">Tier6LuV</button>
  </li>
  <li>
    <button id="Tier7ZPM.html">Tier7ZPM</button>
  </li>
  <li>
    <button id="Tier8UV.html">Tier8UV</button>
  </li>
  <li>
    <button id="Tier9UHV.html">Tier9UHV</button>
  </li>
  <li>
    <button id="Tier10UEV.html">Tier10UEV</button>
  </li>
  <li>
    <button id="Tier11UIV.html">Tier11UIV</button>
  </li>
  <li>


42

In [4]:
str = """
        document.getElementById('%s').addEventListener('click', function () {
            document.getElementById('contentFrame').src = '%s';
        });
"""
for i in os.listdir("html"):
    if "html" in i:
        print(str % (i, "html/" + i))


        document.getElementById('AdeptThaumaturgy.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/AdeptThaumaturgy.html';
        });


        document.getElementById('AndSoItBegins.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/AndSoItBegins.html';
        });


        document.getElementById('AppliedEnergisti.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/AppliedEnergisti.html';
        });


        document.getElementById('BasicAutomation.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/BasicAutomation.html';
        });


        document.getElementById('BeeBreeding.html').addEventListener('click', function () {
            document.getElementById('contentFrame').src = 'html/BeeBreeding.html';
        });


        document.getEl